# Helpers borrowed from [Peter Norvig](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [1]:
import re
import math
from collections import OrderedDict, Counter, defaultdict, namedtuple, deque

from itertools   import permutations, combinations, chain, takewhile, \
cycle, product, islice, zip_longest, count as count_from

from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = 'input_day{}.txt'.format(day)
    return open(filename).read().strip()

cat = ''.join
origin = (0,0)

def array(lines):
    "Convert an iterable of lines into a 2-D array. If lines is a str, do splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return [mapt(atom, line.replace(",", " ").split()) for line in lines]

def atom(token):
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)
            
def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def cityblock_distance(p, q=origin): 
    "Manhatten distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def distance(p, q=origin): 
    "Hypotenuse distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s


# [Day 1](https://adventofcode.com/2017/day/1)

In [2]:
def captcha(input):
    ints = mapt(int, input)
    left = (2*ints)[1:][:len(ints)]
    pairs = zip(ints, left)
    same = filter(lambda p: p[0] == p[1], pairs)
    return sum(map(lambda p: p[0], same))

assert captcha('1122') == 3
assert captcha('1111') == 4
assert captcha('1234') == 0
assert captcha('91212129') == 9

captcha(Input(1))

1144

In [3]:
def captcha_half(input):
    ints = mapt(int, input)
    left = (2*ints)[int(len(ints)/2):][:len(ints)]
    pairs = zip(ints, left)
    same = filter(lambda p: p[0] == p[1], pairs)
    return sum(map(lambda p: p[0], same))

assert captcha_half('1212') == 6
assert captcha_half('1221') == 0
assert captcha_half('123425') == 4
assert captcha_half('123123') == 12
assert captcha_half('12131415') == 4

captcha_half(Input(1))

1194

# [Day 2](https://adventofcode.com/2017/day/2)

In [4]:
def spreadsheet_checksum(input):
    return sum([max(row) - min(row)
                for row in array(input)
               ])

spreadsheet_checksum(Input(2))

45351

In [5]:
def spreadsheet_checksum2(input):
    return sum([p[0] // p[1]
         for row in array(input)
         for p in permutations(row, 2) 
         if p[0] > p[1] 
         and p[0] % p[1] == 0])
        
spreadsheet_checksum2(Input(2))

275

# [Day 3](https://adventofcode.com/2017/day/3)

In [6]:
position = 277678

a = math.ceil((math.sqrt(position)))
if a % 2 == 0:
    a += 1
s = a * a - 4 * (a - 1) + 1
h = (a - 1) // 2
d = abs(((position - s + 1) % (a - 1)) - h)
h + d

475

In [7]:
def populate(m, p, f):
    cx, cy = p
    px = cx - 1
    py = cy
    
    while (cx - 1, cy) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cy += 1
        
    while (cx, cy - 1) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cx -= 1
        
    while (cx + 1, cy) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cy -= 1
        
    while (cx, cy + 1) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cx += 1
        
    return (cx, cy)

def mem_find(cell):
    n = (1, 0)
    mem = OrderedDict([((0, 0), 1)])
    
    while not cell in mem.values():
        n = populate(mem, n, lambda m, p, c: m[p] + 1)
        
    for p, c in mem.items():
        if c == cell:
            return p

p = mem_find(position)
cityblock_distance(p)


475

In [8]:
def neigh(m, p, c):
    return sum([m[n] for n in neighbors8(c) if n in m])
        
def mem_neigh_find(cell):
    n = (1, 0)
    mem = OrderedDict([((0, 0), 1)])
    
    while True:
        n = populate(mem, n, neigh)
        for p, c in mem.items():
            if c > cell:
                return c
        
mem_neigh_find(position)

279138

Different imlpementation based on complex numbers and a generator


In [9]:
U, D, R, L = 1j, -1j, 1, -1
turns = {R:U, U:L, L:D, D:R}

Cell = namedtuple('Cell', 'pos, dir, value')

def cdistance(point): 
    return abs(point.real) + abs(point.imag)

def cells(score):
    pos = 0
    direction = D
    v = 1
    occupied = {pos: v}
    
    while True:
        yield Cell(pos, direction, v)
        if not pos + turns[direction] in occupied:
            direction = turns[direction]
        
        pos += direction
        v = score(pos, v, occupied)
        occupied[pos] = v
            
def mem_find2():            
    for c in cells(lambda p, v, o: v + 1):
        if(c.value >= position):
            return int(cdistance(c.pos))

mem_find2()

475

In [10]:
def neigh8_c(p):
    return (p+R, p+L, p+U, p+D, p+R+U, p+R+D, p+L+U, p+L+D)

def mem_neigh_find2():            
    for c in cells(lambda p, v, o: sum([o[n] for n in neigh8_c(p) if n in o])):
        if(c.value >= position):
            return c.value

mem_neigh_find2()

279138

# [Day 4](https://adventofcode.com/2017/day/4)

In [11]:
def no_duplicates(passphrases):
    valid = []
    for passphrase in passphrases:
        if len(passphrase) == len(set(passphrase)):
            valid.append(passphrase)
            
    return valid
    
valid = no_duplicates(array(Input(4)))
len(valid)

466

In [12]:
def no_anagrams(passphrases):
    valid = []
    for passphrase in passphrases:
        ordred = mapt(cat, mapt(sorted, passphrase))
        if len(ordred) == len(set(ordred)):
            valid.append(passphrase)
        
    return valid
    
ana = no_anagrams(valid) 
len(ana)

251

# [Day 5](https://adventofcode.com/2017/day/5)

In [13]:
jumps = list(map(int, Input(5).split()))
pos = 0
steps = 0

while(pos < len(jumps)):
    p = pos
    pos += jumps[pos]
    jumps[p] += 1
    steps += 1
    
steps

358131

In [14]:
jumps = list(map(int, Input(5).split()))
pos = 0
steps = 0

while(pos < len(jumps)):
    p = pos
    pos += jumps[pos]
    if jumps[p] > 2:
        jumps[p] -= 1
    else:
        jumps[p] += 1  
    steps += 1
    
steps

25558839

# [Day 6](https://adventofcode.com/2017/day/6)

In [15]:
def get_cycles(banks):
    seen = set()

    while not tuple(banks) in seen:
        seen.add(tuple(banks))
        blocks = max(banks)
        i = banks.index(blocks)
        banks[i] = 0

        while blocks > 0:
            i += 1
            i %= len(banks)
            banks[i] += 1
            blocks -= 1

    return len(seen), banks

assert get_cycles([0, 2, 7, 0])[0] == 5

banks = list(array(Input(6))[0])
assert len(banks) == 16
cycles, end_banks = get_cycles(banks)
cycles

11137

In [16]:
loop, _ = get_cycles(end_banks)
loop

1037

# [Day 7](https://adventofcode.com/2017/day/7)

In [17]:
reports = array(Input(7))

def create_tree(reports):
    seen = set()
    children = set()
    tree = {}

    for report in reports:
        name = report[0]
        ch = report[3:]
        weight = int(report[1][1:-1])
        seen.add(name)
        if len(report) > 3:
            children |= set(ch)
            tree[name] = (weight, ch)
        else:
            tree[name] = (weight, [])
        
    return seen.difference(children).pop(), tree

root, tower = create_tree(reports)
root

'gynfwly'

In [18]:
def subtower_weight(start):
    stack = [start]
    weight = 0
    
    while stack:
        name = stack.pop()
        weight += tower[name][0]
        stack.extend(tower[name][1])
    
    return weight

stack = [root]
while stack:
    name = stack.pop()
    weights = {c: subtower_weight(c) for c in tower[name][1]}
    lc = Counter(weights.values()).most_common()[-1]

    if lc[1] == 1:
        diff = lc[0] - Counter(weights.values()).most_common()[0][0]
        stack.append([x for x, y in weights.items() if y == lc[0]][0])
        
tower[name][0] - diff

1526

# [Day 8](https://adventofcode.com/2017/day/8)

In [19]:
instructions = array(Input(8).split('\n'))

def interpret(instructions):
    registers = {}
    max_r = 0
    
    for i in instructions:
        out, op, v, cr = i[0], i[1], i[2], i[4]
        
        if not cr in registers:
            registers[cr] = 0
        if not out in registers:
            registers[out] = 0
            
        exp = str(registers[cr]) + cat(map(str, i[5:]))
            
        if eval(exp):
            if op == 'inc':
                registers[out] += int(v)
            else:
                registers[out] -= int(v)
                
        if registers[out] > max_r:
            max_r = registers[out]

    return max_r, registers
        
max_r, registers = interpret(instructions)
max(registers.values())

6012

In [20]:
max_r

6369

# [Day 9](https://adventofcode.com/2017/day/9)

In [21]:
tests = [
    ("{{{},{},{{}}}}", 16),
    ("{{<!!>},{<!!>},{<!!>},{<!!>}}", 9),
    ("{{<ab>},{<ab>},{<ab>},{<ab>}}", 9),
    ("{{<a!>},{<a!>},{<a!>},{<ab>}}", 3)
]

def groups(stream):
    stack, groups = [], []
    garbage, ignore = False, False
    score, garbage_size = 0, 0
    
    for i in range(len(stream)):
        if not garbage:
            if stream[i] == '{':
                stack.append(i)
            elif stream[i] == '}':
                score += len(stack)
                groups.append(stream[stack.pop():i+1])
            elif stream[i] == '<':
                garbage = True
        else:
            if not ignore:
                if stream[i] == '!':
                    ignore = True
                elif stream[i] == '>':
                    garbage = False
                else:
                    garbage_size += 1 
            else:
                ignore = False
            
    return score, garbage_size
  
for stream, score in tests:
    s, _ = groups(stream)
    assert s == score
    
groups(Input(9))

(8337, 4330)

# [Day 10](https://adventofcode.com/2017/day/10)

In [22]:
def split(li, b, l):
    b %= len(li)
    e = (b + l) % len(li)
    if e < b:
        return li[b:] + li[:e], li[e:b]
    else:
        return li[b:e], li[e:] + li[:b]

def knot_hash(input, sl, repeat=1):
    skip, pos = 0, 0
    hash = [i for i in range(sl)]
    
    for _ in range(repeat):
        for i in input:
            a, b = split(hash, pos, i)
            hash = list(reversed(a)) + b
            hash = hash[sl - pos:] + hash[:sl - pos]

            pos = (pos + i + skip) % sl
            skip += 1

    return hash

input = "63,144,180,149,1,255,167,84,125,65,188,0,2,254,229,24"
test = knot_hash([3, 4, 1, 5], 5)
assert test[0] * test[1] == 12
out1 = knot_hash([int(v) for v in input.split(',')], 256)
out1[0] * out1[1]

4480

In [23]:
from functools import reduce
from binascii import hexlify

def sparse(input):
    bytes = [ord(c) for c in input] + [17, 31, 73, 47, 23]
    sparse = knot_hash(bytes, 256, 64)
    dense = []
    for s in [sparse[i:i+16] for i in range(0, 256, 16)]:
        dense.append(reduce(lambda a,x: a^x, s))
    return str(hexlify(bytearray(dense)))[2:-1]
    
assert sparse("") == 'a2582a3a0e66e6e86e3812dcb672a272'
assert sparse("AoC 2017") == '33efeb34ea91902bb2f59c9920caa6cd'
assert sparse("1,2,3") == '3efbe78a8d82f29979031a4aa0b16a9d'
assert sparse("1,2,4") == '63960835bcdc130f0b66d7ff4f6a5a8e'
sparse(input)

'c500ffe015c83b60fad2e4b7d59dabc4'

# [Day 11](https://adventofcode.com/2017/day/11)

In [24]:
path = Input(11).split(',')

dirs = {'n':  (1, -1, 0),
        'ne': (0, -1, 1),
        'se': (-1, 0, 1),
        's':  (-1, 1, 0),
        'sw': (0, 1, -1),
        'nw': (1, 0, -1)
        }

def hex_dist(point):
    x, y, z = point
    return (abs(x) + abs(y) + abs(z)) // 2
            
def hex_find(path):
    maxp = (0, 0, 0)
    goal = (0, 0, 0)
    for d in path:
        dx, dy, dz = dirs[d]
        goal = (goal[0]+dx, goal[1]+dy, goal[2]+dz)
        if hex_dist(goal) > hex_dist(maxp):
            maxp = goal

    return hex_dist(goal), hex_dist(maxp)

hex_find(path)


(643, 1471)

# [Day 12](https://adventofcode.com/2017/day/12)

In [25]:
test_input = """
0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5
""".strip()

def get_entries(input):
    entries = {entry[0]: set(entry[2:]) for entry in array(input)}         
    return entries

def group(pid, entries):

    stack = [pid]
    g = frozenset()

    while stack:
        pid = stack.pop()
        stack.extend(list(entries[pid].difference(g)))
        g |= entries[pid]

    return g

test_entries = get_entries(test_input)
input_entries = get_entries(Input(12))
    
assert len(group(0, test_entries)) == 6
len(group(0, input_entries))

380

In [26]:
def total(entries):
    groups = set(group(pid, entries) for pid in entries.keys())
    return len(groups)
        
assert total(test_entries) == 2
%time total(input_entries)

CPU times: user 996 ms, sys: 0 ns, total: 996 ms
Wall time: 998 ms


181

# [Day 13](https://adventofcode.com/2017/day/13)

In [27]:
test_input = '''
0: 3
1: 2
4: 4
6: 4
'''.strip()

def get_severity(firewall, delay=0):
    severity = 0
    caught = 0
    s = max(firewall.keys())+1
    for p in range(s):
        if p in firewall:
            if (delay + p) % (2 * (firewall[p] - 1)) == 0:
                severity += p*firewall[p]
                caught += 1
            
    return severity, caught

test_firewall = {d:r for d,r in array(test_input.replace(':', ''))}
firewall = {d:r for d,r in array(Input(13).replace(':', ''))}
assert get_severity(test_firewall)[0] == 24
get_severity(firewall)[0]

2508

In [28]:
%%time
assert get_severity(test_firewall, 10)[0] == 0

delay = 0
while True:
    _, c = get_severity(firewall, delay)
    if c == 0:
        break
    delay += 1
delay

CPU times: user 43.4 s, sys: 0 ns, total: 43.4 s
Wall time: 43.4 s


# [Day 14](https://adventofcode.com/2017/day/14)

In [29]:
def get_grid(input):
    row = lambda x: cat([format((int(d,16)), '04b') for d in sparse(x)])
    return [row(input + '-' + str(i)) for i in range(128)]

test_grid = get_grid("flqrgnkx")
assert Counter(cat(test_grid))['1'] == 8108
grid = get_grid("ljoxqyyw")
Counter(cat(grid))['1']

8316

In [30]:
def regions(grid):
    visited = set()
    cells = [(x, y) for x in range(128) for y in range(128)]

    regs = 0
    dmap = [['.']*128 for _ in range(128)]
    for c in cells:
        x, y = c
        if grid[y][x] == '1' and not c in visited:
            regs += 1
            visited |= set([c])
            stack = [c]
            while stack:
                cc = stack.pop()
                n = [x for x in filter(lambda p: X(p) >=0 and Y(p) >= 0
                           and X(p) <= 127 and Y(p) <= 127
                           and grid[Y(p)][X(p)] == '1'
                           and not p in visited,
                           neighbors4(cc))]
                stack.extend(n)
                visited |= set(n)

    return regs

assert regions(test_grid) == 1242
regions(grid)

1074

# [Day 15](https://adventofcode.com/2017/day/15)

In [31]:
import itertools

fA = 16807
fB = 48271

def gen(s, f, crit=None):
    while True:
        s = (s * f) % 2147483647
        if crit is None or s % crit == 0:
            yield s

def total_matches(ga, gb, mx=40000000):
    total = 0
    for a, b in itertools.islice(zip(ga, gb), mx):
        if (a & 0xFFFF) == (b & 0xFFFF):
            total += 1
    return total

#assert total_matches(gen(65, fA), gen(8921, fB)) == 588
total_matches(gen(289, fA), gen(629, fB))

638

In [32]:
total_matches(gen(289, fA, 4), gen(629, fB, 8), 5000000)

343

# [Day 16](https://adventofcode.com/2017/day/16)

In [33]:
def prgms(l):
    return [chr(i) for i in range(ord('a'), ord(l)+1)]

def dance(moves, s):
    for m in moves:
        if m[0] == 's':
            s = s[len(s)-int(m[1:]):] + s[:len(s)-int(m[1:])]
        elif m[0] == 'x':
            p1, p2 = map(int, m[1:].split("/"))
            s[p1], s[p2] = s[p2], s[p1]
        elif m[0] == 'p':
            p1, p2 = m[1:].split("/")
            p1 = cat(s).find(p1)
            p2 = cat(s).find(p2)
            s[p1], s[p2] = s[p2], s[p1]
    return s

assert cat(dance(['s1', 'x3/4', 'pe/b'], prgms('e'))) == 'baedc'
cat(dance(array(Input(16))[0], prgms('p')))

'ebjpfdgmihonackl'

In [34]:
def dance_rep(moves, s, i):
    start = cat(s)
    togo = i
    while togo > 0:
        s = dance(moves, s)
        if cat(s) == start:
            togo = i % (i - togo + 1)
        else:
            togo -= 1
    return s
        
assert cat(dance_rep(['s1', 'x3/4', 'pe/b'], prgms('e'), 2)) == 'ceadb'
cat(dance_rep(array(Input(16))[0], prgms('p'), 1000000000))

'abocefghijklmndp'

# [Day 17](https://adventofcode.com/2017/day/17)

In [35]:
def spinlock(step):
    state, pos, count = [0], 0, 0
    while True:
        yield state[(pos+1) % len(state)]
        pos = (pos + step) % len(state) + 1
        count += 1
        state.insert(pos, count)

def spin(spinlock, times=2017):
    for _ in range(times):
        next(spinlock)
    return next(spinlock)
   
assert spin(spinlock(3)) == 638
spin(spinlock(386))

419

In [36]:
def spinlock2(step):
    pos, count, out = 0, 0, 0
    while True:
        yield out
        pos = ((pos + step) % (count + 1)) + 1
        count += 1
        if pos == 1:
            out = count
        
%time spin(spinlock2(386), 50000000)

CPU times: user 13.6 s, sys: 4 ms, total: 13.6 s
Wall time: 13.6 s


46038988

# [Day 18](https://adventofcode.com/2017/day/18)

In [37]:
from collections import deque

test_duet = array("""
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
""".strip())

def prgm(inst, inp, out, pid=0):
    regs = defaultdict(int)
    regs['p'] = pid
    i, s, l = 0, 0, 0
    ev = lambda a: regs[a] if isinstance(a, str) else a
    
    while i >= 0 and i < len(inst):
        cmd = inst[i]
        
        if cmd[0] == 'snd':
            out.append(ev(cmd[1]))
            l = ev(cmd[1])
            s += 1
        elif cmd[0] == 'set':
            regs[cmd[1]] = ev(cmd[2])
        elif cmd[0] == 'add':
            regs[cmd[1]] += ev(cmd[2])
        elif cmd[0] == 'mul':
            regs[cmd[1]] *= ev(cmd[2])
        elif cmd[0] == 'mod':
            regs[cmd[1]] %= ev(cmd[2])
        elif cmd[0] == 'rcv':
            while len(inp) == 0:
                yield l, s
            regs[cmd[1]] = inp.popleft()
        elif cmd[0] == 'jgz' and ev(cmd[1]) > 0:
            i += ev(cmd[2])
            continue
        i += 1
        
def run(inst):
    inp, out = deque(), deque()
    p = prgm(inst, inp, out)
    return next(p)

assert run(test_duet)[0] == 4
run(array(Input(18)))[0]

4601

In [38]:
def run2(inst):
    inp, out = deque(), deque()
    p0 = prgm(inst, inp, out, 0)
    p1 = prgm(inst, out, inp, 1)
    
    while True:
        next(p0)
        s1 = next(p1)
        if len(inp)==0 and len(out)==0:
            break
        
    return s1
    
run2(array(Input(18)))[1]

6858

# [Day 19](https://adventofcode.com/2017/day/19)

In [39]:
ppath = open("input_day19.txt").read().splitlines()

def traverse(path):
    p = (path[0].index('|'), 0) #starting point
    d = (0, 1) #direction == down
    pp, s, steps = p, "", 0
    
    while True:
        np = (X(p) + X(d), Y(p) + Y(d))
        steps += 1
        
        if path[Y(np)][X(np)] == ' ':
            nn = list(filter(lambda a: a != pp and path[Y(a)][X(a)] != ' ', neighbors4(p)))
            if len(nn) == 0: break #blind alley?
            assert len(nn) == 1
            np = nn[0]
            d = (X(np) - X(p), Y(np) - Y(p))

        if path[Y(np)][X(np)].isalpha():
            s += path[Y(np)][X(np)]
            
        pp, p = p, np
        
    return s, steps
    
traverse(ppath)[0]

'KGPTMEJVS'

In [40]:
traverse(ppath)[1]

16328

# [Day 20](https://adventofcode.com/2017/day/20)

In [41]:
def particles(input):
    parts = {}
    
    for i, l in enumerate(input.splitlines()):
        p = mapt(int, re.findall(r"-?\d+", l))
        parts[i] = (p[0:3], p[3:6], p[6:9])
    return parts

def evolve(parts):
    for k,vl in parts.items():
        p,v,a = vl
        nv = (v[0] + a[0], v[1] + a[1], v[2] + a[2])
        np = (p[0] + nv[0], p[1] + nv[1], p[2] + nv[2])
        parts[k] = (np, nv, a)
        
def find_closest(parts, process=None):
    pdist = lambda p: sum([abs(a) for a in p])
    psort = lambda pc: sorted(pc, key=lambda p: pdist(pc.get(p)[0]))
    s1 = psort(parts)
    
    while True:
        if process:
            process(parts)
            
        evolve(parts)
        s2 = psort(parts)  # check if no particles pass each other
        if s1 == s2: break # and treat this as 'steady state'
        s1 = s2
    
    return s2[0], len(parts)

p = particles(Input(20))
find_closest(p)[0]

125

In [42]:
def pproces(parts):
    c = Counter([v[0] for k,v in parts.items()])
    cols = [a for a in c if c[a] > 1]
    ci = [k for k,v in parts.items() if v[0] in cols]
        
    for i in ci:
        parts.pop(i)

p = particles(Input(20))
find_closest(p, pproces)[1]

461

# [Day 21](https://adventofcode.com/2017/day/21)

In [43]:
def tiles(tile):
    rot = lambda s: '/'.join(map(cat, zip(*(s.split('/'))[::-1])))
    flip = lambda s: '/'.join([l[::-1] for l in s.split('/')])
    
    ftile = flip(tile)
    t = {tile, ftile}
    for _ in range(3):
        tile = rot(tile)
        ftile = rot(ftile)
        t |= {tile, ftile}
    return t

def rules(input):
    rules = {}
    for l in input.strip().splitlines():
        k, v = (l.replace(' ', '').split('=>'))
        rules[k] = v
        for t in tiles(k):
            rules[t] = v
            
    return rules

def stitch(tiles):
    tt = [t.split('/') for t in tiles]
    s = []
    for r in zip(*tt):
        s += [''.join(r)]
    return s

def unfold(rules):
    pat = \
    """
    .#.
    ..#
    ###
    """
    
    while True:
        sm = [r.strip() for r in pat.strip().splitlines()]
        ps = len(sm)
        assert ps == sum(map(len,sm))/ps

        m = []
        step = 2 if (ps % 2) == 0 else 3
        size = ps // step
        for y in range(0, ps, step):
            for x in range(0, ps, step):
                m += ['/'.join([sm[yy][x:x+step] for yy in range(y, y+step)])]

        for i, t in enumerate(m):
            m[i] = rules[t]

        pat = ""
        for y in range(0, size*size, size):
            for l in stitch(m[y:y+size]):
                pat += l + '\n'
        yield pat
        
tr = rules(Input(21))
g = unfold(tr)
t = [t for t in islice(g, 0, 5)][-1]
print(t)
sum(1 if x == '#' else 0 for x in t)

#.##..#.##..#.##.#
.#.....#.....#..#.
#.....#.....#..#..
..#..#..#..#..##.#
..#..#..#..#..#.#.
#..#..#..#..#..#..
#.##..#.##....##.#
.#.....#....#.#.#.
#.....#.......##..
..#..#..#..#..#...
..#..#..#..##.##..
#..#..#..#....#..#
#.##.##.##..#.##..
.#..#..#.....#....
#..#..#.....#.....
..##.#..#..#..#..#
..#.#...#..#..#..#
#..#..#..#..#..#..



110

In [44]:
g = unfold(tr)
%time t = [t for t in islice(g, 0, 18)][-1]
sum(1 if x == '#' else 0 for x in t)

CPU times: user 1.64 s, sys: 4 ms, total: 1.65 s
Wall time: 1.65 s


1277716

# [Day 22](https://adventofcode.com/2017/day/22)

In [45]:
U, D, R, L = -1j, 1j, 1, -1
left  = {R:U, U:L, L:D, D:R}
right = {R:D, D:L, L:U, U:R}
rev   = {R:L, L:R, U:D, D:U}
    
def grid(input):
    grid  = {}
    size = len(input.splitlines())
    for y, l in enumerate(input.splitlines()):
        assert len(l) == size
        for x, n in enumerate(l):
            if n == '#':
                grid[complex(x,y)] = 'I'
                
    return grid, size

def infection(pos, grid):
    inf = 0
    d = U
    
    while True:
        if pos in grid:
            d = right[d]
            grid.pop(pos)
        else:
            d = left[d]
            grid[pos] = 'I'
            inf += 1
        yield inf
        pos += d
        
def infections(input, rep, i=infection):
    g, size = grid(input)
    pos = complex(size // 2, size // 2)
    sim = i(pos, g)
    return [s for s in islice(sim, 0, rep)][-1]

ig = """..#
#..
...""".strip()

assert infections(ig, 70) == 41
assert infections(ig, 10000) == 5587

infections(Input(22), 10000)

5450

In [46]:
def infection2(pos, grid):
    inf = 0
    d = U
    
    while True:
        if pos in grid:
            if grid[pos] == 'I':
                d = right[d]
                grid[pos] = 'F'
            elif grid[pos] == 'F':
                d = rev[d]
                grid.pop(pos)
            elif grid[pos] == 'W':
                grid[pos] = 'I'
                inf += 1
        else:
            d = left[d]
            grid[pos] = 'W'

        yield inf
        pos += d

assert infections(ig, 100, infection2) == 26
assert infections(ig, 10000000, infection2) == 2511944
%time infections(Input(22), 10000000, infection2)

CPU times: user 5.52 s, sys: 52 ms, total: 5.57 s
Wall time: 5.58 s


2511957

# [Day 23](https://adventofcode.com/2017/day/23)

In [47]:
def prgm(inst, a = 0):
    regs = defaultdict(int)
    regs['a'] = a
    i, mc = 0, 0
    ev = lambda a: regs[a] if isinstance(a, str) else a
    
    while i >= 0 and i < len(inst):
        cmd = inst[i]
        if cmd[0] == 'set':
            regs[cmd[1]] = ev(cmd[2])
        elif cmd[0] == 'sub':
            regs[cmd[1]] -= ev(cmd[2])
        elif cmd[0] == 'mul':
            regs[cmd[1]] *= ev(cmd[2])
            mc += 1
        elif cmd[0] == 'jnz' and ev(cmd[1]) != 0:
            i += ev(cmd[2])
            continue
        i += 1
    return mc
        
instr = array(Input(23))
prgm(instr)

4225

In [48]:
def optim(b, c):
    h = 0
    while True:
        f = 1
        for i in range(2, b):
            if (b % i) == 0:
                f = 0
                break
        if f == 0:
            h += 1
        if b == c:
            break

        b += 17
    return h

b = 67
c = 67
b = b*100+100000
c = b + 17000
optim(b, c)

905

# [Day 24](https://adventofcode.com/2017/day/24)

In [49]:
def components(input):
    c = []
    for l in input.strip().splitlines():
        i, o = map(int, l.split('/'))
        c.append((i, o))
    return c

def strength(bridge):
    return sum([i + o for i, o in bridge])

def build(b, p, cp):
    mat = [(i, o) for i, o in cp if i == p or o == p]
    r = []
    for c in mat:
        rc = [x for x in cp if x != c]
        np = c[1] if c[0] == p else c[0] 
        r.append((b + [c], np, rc))
    return r
        
def bridges(cnts):
    stack, bdges = [], []
    for c in [(i, o) for i, o in cnts if i == 0 or o == 0]:
        stack.append(([c], c[1] if c[0] == 0 else c[0], [x for x in cnts if x != c]))
        
    while stack:
        b, p, c = stack.pop()
        nb = build(b, p, c)
        if nb:
            stack.extend(nb)
        else:
            bdges.append(b)  
    return bdges

tc = components("""
0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10
""")

assert max(map(strength, bridges(tc))) == 31
c = components(Input(24))
b = bridges(c)
max(map(strength, b))

1859

In [50]:
longest = max(map(len, b))
max(map(strength, [x for x in b if len(x) == longest]))

1799